# Subtask 1

In [1]:
import pandas as pd

amh = pd.read_csv('../dev_phase/subtask1/train/amh.csv')

### Amharic

In [2]:
print('Total:', len(amh), 'Polarization:', len(amh[amh['polarization'] == 1]))
amh.head()

Total: 3332 Polarization: 2518


,id,text,polarization
0,amh_6713e86058c564a4b874dd62227b7fbc,ወፈፌ ቀን አልፎ ዕብድ ቀን ሲመጣ፣ ሰይጣን ፀበል ገብቶ ሰው ከሰይጣን ወጣ።,1
1,amh_50c28694a056e584ee76da86ed1875ef,የአማራ ባንክ የምስጋና እና የዕውቅና መርሐ-ግብር አማራ ባንክ ከባንክ ባ...,0
2,amh_3fe8faab2cf4c60b9bed28eed5f1c864,ራያ ግንባር ጎብዬ መከላከያ፣ የአማራ ልዩ ሀይል እና የምስራቅ አማራ ፋኖ...,0
3,amh_9b7badaab07f0e9e3dd77b99894bbb9d,ሩሲያ ቴርሞባሪክ ቦምብ ከመጠቀሟ ጋር ተያይዞ መላው አውሮፓ ላይ ጭንቀት ...,0
4,amh_be6049aa059a1ccfce6077d0cb8fd9f2,ዮኒ ማኛ ለማኝ ስግብግብ ሚዲዳዎችን ዱቄት በዱቄት። እውነት ነው በጣም ያ...,1


## Classification

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

def print_evaluation(y_test, y_pred):
    f1_macro = f1_score(y_test, y_pred, average='macro')
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1_binary = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print(f"F1-score (Macro) = {f1_macro:.4f}")
    print(f"Accuracy = {accuracy:.4f}")
    print(f"Precision = {precision:.4f}")
    print(f"Recall = {recall:.4f}")
    print(f"F1-score (Binary) = {f1_binary:.4f}")

    print(f"Confusion Matrix:\n{cm}")

### TF-IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()

TfidfVectorizer:

F1-score (Macro) = 0.4592
Accuracy = 0.7451
Precision = 0.7439
Recall = 0.9980
F1-score (Binary) = 0.8524
Confusion Matrix:
[[  6 169]
 [  1 491]]


### Count Vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(binary=False)


CountVectorizer:

F1-score (Macro) = 0.6113
Accuracy = 0.7691
Precision = 0.7817
Recall = 0.9533
F1-score (Binary) = 0.8590
Confusion Matrix:
[[ 44 131]
 [ 23 469]]


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
# from transformers import AutoTokenizer
from transformers import PreTrainedTokenizerFast

# tokenizer = AutoTokenizer.from_pretrained("rasyosef/bert-amharic-tokenizer")
tokenizer = PreTrainedTokenizerFast.from_pretrained("Hailay/geez-tokenizer")

def amharic_tokenizer(text):
    return tokenizer.tokenize(text)

vect = TfidfVectorizer(tokenizer=amharic_tokenizer, lowercase=False)

pipe = Pipeline([
    ("vect", vect),
    ("clf", LogisticRegression(class_weight="balanced", random_state=0, max_iter=500))
])

param_grid = {
    "vect__ngram_range": [(1,1), (1,2)],
    "clf__C": [0.1, 1, 10],
}

X_train, X_test, y_train, y_test = train_test_split(amh['text'], amh['polarization'], test_size=0.2, random_state=42)

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, scoring="f1_macro", n_jobs=-1, verbose=2)
grid.fit(X_train, y_train)

print("Beste Parameter:", grid.best_params_)
print("Bestes F1 (macro):", grid.best_score_)

y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 6 candidates, totalling 30 fits


c:\Users\gcreg\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Beste Parameter: {'clf__C': 1, 'vect__ngram_range': (1, 2)}
Bestes F1 (macro): 0.6739103835147127
              precision    recall  f1-score   support

           0       0.59      0.49      0.53       175
           1       0.83      0.88      0.85       492

    accuracy                           0.78       667
   macro avg       0.71      0.68      0.69       667
weighted avg       0.77      0.78      0.77       667

